In [1]:
import random
from huggingface_hub import InferenceClient
import numpy as np
import json

In [ ]:
def list_to_string(list):
    string='['
    for i in list:
        string=string+i
        string=string+', '
    string=string[:-2]+']'
    return string
def string_to_list(string):
    s=''.join(string)
    s=s.split('\n')
    list=[]
    for i in range(len(s)):
        if len(s[i])>3:
            if s[i][0].isdigit() or s[i][1].isdigit() or s[i][2].isdigit():
                a=False
                b=0
                for j in range(len(s[i])):
                    if s[i][j].isdigit():
                        a=True
                    if not s[i][j].isdigit() and a:
                        b=j
                        break
                list.append('"'+s[i][b+1:]+' ",\n')
    return list
        

def load(genre):
    prompts=[]
    f='prompt/'+genre+'_start.txt'
    f=open(f,'r')
    for prompt in f:
        prompts.append(prompt)
    return prompts


def write_to_file(prompt,genre):
    f='Dataset/'+genre+'_prompts.txt'
    f=open(f,'a')
    f.write(prompt)
    f.close()

In [ ]:
mistral_client = InferenceClient(model="mistralai/Mistral-7B-Instruct-v0.2", token="")

genre='narrative'

n_p = 12 #Number of prompts used to generate new prompts

current_prompts=load(genre)

prompts = []
seeds=[]
texts=[]

dataset = []

# Function to append data to the dataset and write to file
def append_to_dataset_and_write(data):
    dataset.append(data)
    with open("Dataset/"+genre+".json", 'w') as file:
        json.dump(dataset, file)

x = np.random.randint(0, 1e7)
#current_prompts=mistral_client.text_generation('Please generate a prompt list inspired by the list below. \n\n ' + list_to_string(current_prompts), max_new_tokens=500, do_sample=True, temperature=1.0, seed=x)
#current_prompts=string_to_list(current_prompts)
while True:
    for prompt in current_prompts:
        x = np.random.randint(0, 1e7)
        text_sample=mistral_client.text_generation(prompt, max_new_tokens=500, do_sample=True, temperature=1.0, seed=x)

        #label = mistral_client.text_generation('Here is a text:\n\n ' + text_sample+'\n\n Output one of these genres best describing the text above. Your output should be a single word. These are the possible genres:\n\n instructional, list, narrative, explanatory, dialogue, code.' , max_new_tokens=50, do_sample=True, temperature=1.0, seed=x)
        #write_to_file(x,prompt,text_sample,genre)
        data ={'prompt':prompt, 'text':text_sample, 'seed':x}
        append_to_dataset_and_write(data)
        prompts.append(prompt)
        seeds.append(x)
        texts.append(text_sample)
        np.save('Dataset/prompts_'+genre,prompts)
        np.save('Dataset/seeds_'+genre,seeds)
        np.save('Dataset/texts_'+genre,texts)
        write_to_file(prompt,genre)
    
    current_prompts = []
    while len(current_prompts) < n_p/2.0:
        current_prompts=mistral_client.text_generation('Please generate a prompt list that elicits similar genre text as the prompts below. \n\n ' + list_to_string(random.sample(prompts,n_p)) + " \n\n Answer only with the enumerated list of new prompts and nothing else.", max_new_tokens=500, do_sample=True, temperature=1.0, seed=x)
        current_prompts=string_to_list(current_prompts)
